In [1]:
#load the library
import numpy as np
import pandas as pd

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
# from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import optuna
import warnings
from tqdm import tqdm
import time
from sklearn.decomposition import TruncatedSVD, PCA
import joblib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
#transfrom categorical data to numbers
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [3]:
#import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [7]:
train.shape

(891, 12)

## Fill NAs

In [8]:
sub.PassengerId.to_list() == test.PassengerId.to_list()

True

In [9]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
## fill age
train.Age = train.Age.fillna(np.mean(train.Age))

In [11]:
train.Cabin = train.Cabin.fillna('Unknown')

In [12]:
train.Embarked = train.Embarked.fillna('Unknown')

In [13]:
train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [14]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [15]:
test.Age = test.Age.fillna(np.mean(train.Age))
test.Fare = test.Fare.fillna(np.mean(test.Fare))

In [16]:
test.Cabin = test.Cabin.fillna('Unknown')

In [17]:
test.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## Split folds

In [18]:
#do k fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [19]:
train['fold'] = -999
for fold_id, (train_index, valid_index) in enumerate(skf.split(X=train, y=train.Survived.values)):
    train.loc[valid_index, 'fold'] = fold_id

In [20]:
train.fold.value_counts()

0    179
1    178
2    178
3    178
4    178
Name: fold, dtype: int64

## Feature Engineering

#### 1. Name Title (Change the name to title and label encoding)

In [21]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [22]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [23]:
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(test['Title'], test['Sex'])

Sex,female,male
Title,,
Col,0,2
Dona,1,0
Dr,0,1
Master,0,21
Miss,78,0
Mr,0,240
Mrs,72,0
Ms,1,0
Rev,0,2


In [24]:
test['Title'] = test['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

In [25]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

train['Title'] = train['Title'].map(title_mapping)
train['Title'] = train['Title'].fillna(0)
test['Title'] = test['Title'].map(title_mapping)
test['Title'] = test['Title'].fillna(0)

In [26]:
# train = train.drop('Name', axis=1)
# test = test.drop('Name', axis=1)

In [27]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,0,1


In [28]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,Unknown,S,3


#### 2.  Cut the Age and Fare column to different bins 

In [29]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,fold,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,1.997755,1.728395
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429,1.415800,1.030039
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,1.000000,1.000000
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200,2.000000,1.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,3.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,4.000000,5.000000


In [30]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Title
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,30.154603,0.447368,0.392344,35.627188,1.741627
std,120.810458,0.841838,12.636666,0.896760,0.981429,55.840500,1.006091
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,29.699118,0.000000,0.000000,14.454200,1.000000
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000,2.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,5.000000


In [31]:
train['Fare'] = pd.qcut(train['Fare'], 10)
test['Fare'] = pd.qcut(test['Fare'], 10)

In [32]:
train['Age'] = pd.qcut(train['Age'], 10, duplicates='drop')
test['Age'] = pd.qcut(test['Age'], 10, duplicates='drop')

In [33]:
type(train['SibSp'][0])

numpy.int64

In [34]:
train['SibSp'] = pd.qcut(train['SibSp'], 4, duplicates='drop')
test['SibSp'] = pd.qcut(test['SibSp'], 4, duplicates='drop')

In [35]:
train['Fare'] = str(train['Fare'])
test['Fare'] = str(test['Fare'])

train['Age'] = str(train['Age'])
test['Age'] = str(test['Age'])

train['SibSp'] = str(train['SibSp'])
test['SibSp'] = str(test['SibSp'])

In [36]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,fold,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,"0 (20.0, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,A/5 21171,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",Unknown,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,"0 (20.0, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,PC 17599,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",C85,C,4,3
2,3,1,3,"Heikkinen, Miss. Laina",female,"0 (20.0, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,STON/O2. 3101282,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",Unknown,S,3,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,"0 (20.0, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,113803,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",C123,S,3,3
4,5,0,3,"Allen, Mr. William Henry",male,"0 (20.0, 24.0]\n1 (32.5, 38.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,373450,"0 (-0.001, 7.55]\n1 (39.688, 77.95...",Unknown,S,0,1


In [37]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,892,3,"Kelly, Mr. James",male,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,330911,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,363272,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,S,3
2,894,2,"Myles, Mr. Thomas Francis",male,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,240276,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,Q,1
3,895,3,"Wirz, Mr. Albert",male,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",0,315154,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,"0 (31.9, 39.0]\n1 (39.0, 48.0]\n...","0 (-0.001, 1.0]\n1 (-0.001, 1.0]\n2 ...",1,3101298,"0 (7.796, 8.05]\n1 (-0.001, 7.64...",Unknown,S,3


## Label Encoding

In [38]:
# train.Name = train.Name.str.extract('([A-Za-z]+)\.', expand=False)
# test.Name = test.Name.str.extract('([A-Za-z]+)\.', expand=False)

In [39]:
train = train.drop('Name', axis=1)
test = test.drop('Name', axis=1)

In [40]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [41]:
train, test = label_encoding(train, test, ['Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', 'Fare', 'SibSp'] )

# Building Model

## LGB

In [44]:
def optuna_lgb(n_trials=100):
    
    def objective(trial):

        params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-5, 10, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-5, 10, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 62),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 0.9),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 0.9),
            "bagging_freq": trial.suggest_int("bagging_freq", 2, 9),
            "min_child_samples": trial.suggest_int("min_child_samples", 33, 93),
            "max_depth": trial.suggest_int("max_depth", 2, 7)
        }

        # initialize oof 
        oof_train = np.zeros((len(train),))

        for i in range(5):
            train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
            train_y = train.query(f'fold!={i}').Survived

            valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
            valid_y = train.query(f'fold=={i}').Survived
            
            lgb_train = lgb.Dataset(train_x,
                                    train_y)

            lgb_eval = lgb.Dataset(valid_x,
                                   valid_y,
                                   reference=lgb_train)

            gbm = lgb.train(params,
                            lgb_train,
                            valid_sets=[lgb_train, lgb_eval],
                            num_boost_round=5000,
                            verbose_eval=-1, 
                            early_stopping_rounds=1000
                            )
            oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
            oof_train[valid_x.index] = gbm.predict(valid_x,
                                                   num_iteration=gbm.best_iteration)
            
            
        return accuracy_score(train.Survived, oof_train.round())
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best_params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }
    best_params.update(study.best_params)
    ## save trained model
    # initialize oof 
    oof_train = np.zeros((len(train),))
    print('*'*50)
    print('Exporting best models')
    print('*' * 50)
    # split folds using RSGKF
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        lgb_train = lgb.Dataset(train_x,
                                train_y)

        lgb_eval = lgb.Dataset(valid_x,
                               valid_y,
                               reference=lgb_train)

        gbm = lgb.train(best_params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_eval],
                        num_boost_round=50000,
                        verbose_eval=-1, 
                        early_stopping_rounds=1001
                        )

        oof_preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
        oof_train[valid_x.index] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)

        joblib.dump(gbm, f'lgb_fold{i}.pkl')

In [ ]:
optuna_lgb() #0.8338945005611672(5 fold)  0.8451178451178452(10 fold)

[I 2021-10-02 20:06:30,993] A new study created in memory with name: no-name-3e6934d0-a518-4e1f-8c05-50467b58a0ce


Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[93]	training's binary_logloss: 0.380899	valid_1's binary_logloss: 0.415107
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[226]	training's binary_logloss: 0.353536	valid_1's binary_logloss: 0.399172
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[131]	training's binary_logloss: 0.359263	valid_1's binary_logloss: 0.458907
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[516]	training's binary_logloss: 0.315719	valid_1's binary_logloss: 0.413806
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:06:32,904] Trial 0 finished with value: 0.8170594837261503 and parameters: {'lambda_l1': 0.4504811473905986, 'lambda_l2': 1.355569361399226, 'num_leaves': 29, 'feature_fraction': 0.5243510836328067, 'bagging_fraction': 0.7409490661146226, 'bagging_freq': 3, 'min_child_samples': 42, 'max_depth': 3}. Best is trial 0 with value: 0.8170594837261503.


Early stopping, best iteration is:
[35]	training's binary_logloss: 0.41142	valid_1's binary_logloss: 0.42258
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[352]	training's binary_logloss: 0.357577	valid_1's binary_logloss: 0.413776
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[579]	training's binary_logloss: 0.336561	valid_1's binary_logloss: 0.389818
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[162]	training's binary_logloss: 0.377119	valid_1's binary_logloss: 0.453438
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[287]	training's binary_logloss: 0.360804	valid_1's binary_logloss: 0.42291
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:06:35,124] Trial 1 finished with value: 0.8159371492704826 and parameters: {'lambda_l1': 0.0029842956381218428, 'lambda_l2': 0.1537110283963997, 'num_leaves': 8, 'feature_fraction': 0.6438285308285523, 'bagging_fraction': 0.5542362596910381, 'bagging_freq': 3, 'min_child_samples': 69, 'max_depth': 5}. Best is trial 0 with value: 0.8170594837261503.


...

Early stopping, best iteration is:
[40]	training's binary_logloss: 0.426543	valid_1's binary_logloss: 0.429732
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[821]	training's binary_logloss: 0.331553	valid_1's binary_logloss: 0.415826
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[645]	training's binary_logloss: 0.346152	valid_1's binary_logloss: 0.391598
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[398]	training's binary_logloss: 0.352786	valid_1's binary_logloss: 0.449658
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[229]	training's binary_logloss: 0.377004	valid_1's binary_logloss: 0.418724
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:10:39,377] Trial 93 finished with value: 0.8215488215488216 and parameters: {'lambda_l1': 0.00021737052645755118, 'lambda_l2': 0.04407263870281769, 'num_leaves': 16, 'feature_fraction': 0.40406579761605055, 'bagging_fraction': 0.7920719536926591, 'bagging_freq': 7, 'min_child_samples': 85, 'max_depth': 3}. Best is trial 83 with value: 0.8271604938271605.


Early stopping, best iteration is:
[45]	training's binary_logloss: 0.421689	valid_1's binary_logloss: 0.427366
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[406]	training's binary_logloss: 0.348859	valid_1's binary_logloss: 0.414901
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[525]	training's binary_logloss: 0.342424	valid_1's binary_logloss: 0.399106
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[235]	training's binary_logloss: 0.359584	valid_1's binary_logloss: 0.454453
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[244]	training's binary_logloss: 0.366191	valid_1's binary_logloss: 0.424079
Training until validation scores don't improve for 1000 rounds


[I 2021-10-02 20:10:41,747] Trial 94 finished with value: 0.8092031425364759 and parameters: {'lambda_l1': 5.964811024142487e-05, 'lambda_l2': 0.0824047086719883, 'num_leaves': 14, 'feature_fraction': 0.8791444554536215, 'bagging_fraction': 0.8017891713917388, 'bagging_freq': 7, 'min_child_samples': 93, 'max_depth': 3}. Best is trial 83 with value: 0.8271604938271605.


Early stopping, best iteration is:
[38]	training's binary_logloss: 0.423317	valid_1's binary_logloss: 0.427616
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[265]	training's binary_logloss: 0.358717	valid_1's binary_logloss: 0.424301
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[450]	training's binary_logloss: 0.343274	valid_1's binary_logloss: 0.38747
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[231]	training's binary_logloss: 0.358163	valid_1's binary_logloss: 0.456244
Training until validation scores don't improve for 1000 rounds


## XGBoost

In [ ]:
#optuna xgboost example
#https://github.com/optuna/optuna-examples/blob/main/xgboost/xgboost_integration.py
import xgboost as xgb
def objective(trial):
    param = {
            "verbosity": 0,
            "objective": "binary:logistic",  #binary:logistic
            "eval_metric": "auc",  #map
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived

        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        
        accuracy = []
        # Add a callback for pruning.
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
        bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
        preds = bst.predict(dvalid)
        pred_labels = np.rint(preds)
        accuracy.append(sklearn.metrics.accuracy_score(valid_y, pred_labels))
        
        
        joblib.dump(bst, f'xgboost_fold{i}.pkl')
    return np.mean(accuracy)

    
if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

## Catboost

In [ ]:
#optuna catboost example
#https://github.com/optuna/optuna-examples/blob/main/catboost/catboost_simple.py
import catboost as cb
def objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
        
        #save the five fold accuracy
        accuracy = []
        
        gbm = cb.CatBoostClassifier(**param)
        gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
        
        #save the best model for each fold 
        joblib.dump(gbm, f'catboost_fold{i}.pkl')
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Logistic and Random Forest and SVC

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = trial.suggest_categorical("classifier", ["LogReg", "SVC", "RandomForest"])
    
    # Step 2. Setup values for the hyperparameters:
    if classifier_name == 'LogReg':
        logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
        classifier_obj = linear_model.LogisticRegression(C=logreg_c)
    elif classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
    else:
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators
        )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'log_randomf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Logistic

In [ ]:
#logistic

#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'LogReg'
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    classifier_obj = linear_model.LogisticRegression(C=logreg_c)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'logistic_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## SVC

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    # Step 2. Setup values for the hyperparameters:
    classifier_name = "SVC"
    svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
    classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'svc_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## RandomForest

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    classifier_name = 'RandomForest'
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
#     rf_min_depth = trial.suggest_int("rf_min_depth", 2, 32, log=True)
    classifier_obj = ensemble.RandomForestClassifier(
        max_depth=rf_max_depth, n_estimators=rf_n_estimators
    )
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        gbm = classifier_obj
        gbm.fit(train_x, train_y)
        preds = gbm.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(gbm, f'rf_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Decision Tree

In [ ]:
#Step 1. Define an objective function to be maximized.
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 612)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 612)
    max_leaf_nodes = int(trial.suggest_int("max_leaf_nodes", 2, 612))
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    DTC = DecisionTreeClassifier(min_samples_split = min_samples_split, 
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion)
        
    for i in range(5):
        train_x = train.query(f'fold!={i}').drop(['fold', 'Survived'], axis=1)
        train_y = train.query(f'fold!={i}').Survived

        valid_x = train.query(f'fold=={i}').drop(['fold', 'Survived'], axis=1)
        valid_y = train.query(f'fold=={i}').Survived
    
        #save the five fold accuracy
        accuracy = []

        DTC.fit(train_x, train_y)
        preds = DTC.predict(valid_x)
        pred_labels = np.rint(preds)
        accuracy.append(accuracy_score(valid_y, pred_labels))
                
        #save the best model for each fold 
        joblib.dump(DTC, f'dt_fold{i}.pkl')
        
    return np.mean(accuracy)

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

## Ensemble Model

In [ ]:
X_train = train.drop(['Survived', 'fold'], axis=1).values
y_train = train[['Survived']].values

In [ ]:
test2 = test.drop(['Survived', 'fold'], axis=1).values

In [ ]:
#get the mean of best fold in lgb   0.8338945005611672  0.77990 selected
lgb_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./lgb_fold{i}.pkl')
    lgb_pred += model.predict(test2) / 5
    
lgb_pred = lgb_pred.round().astype(int)

In [ ]:
#get the mean of best fold in xgboost  0.8314606741573034  0.78468 selected
from xgboost import XGBClassifier
xgboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./xgboost_fold{i}.pkl')
    model = XGBClassifier()
    model.fit(X_train, y_train)
    xgboost_pred += model.predict_proba(test2)[:,1] / 5   #predict probability [0, 1]
    
xgboost_pred = xgboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in catboost  0.8426966292134831  nice  0.77990 selected
catboost_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./catboost_fold{i}.pkl')
    catboost_pred += model.predict(test2) / 5
    
catboost_pred = catboost_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic  0.8426966292134831 0.78947 selected nice
logistic_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./logistic_fold{i}.pkl')
    logistic_pred += model.predict(test2) / 5
    
logistic_pred = logistic_pred.round().astype(int)

In [ ]:
#get the mean of best fold in logistic/randomForest/svc  
#randomForest win 0.8314606741573034
log_ran_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./log_randomf_fold{i}.pkl')
    log_ran_pred += model.predict(test2) / 5
    
log_ran_pred = log_ran_pred.round().astype(int)

In [ ]:
#get the mean of best fold in svc  0.7528089887640449    0.66985
svc_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./svc_fold{i}.pkl')
    svc_pred += model.predict(test2) / 5
    
svc_pred = svc_pred.round().astype(int)

In [ ]:
#get the mean of best fold in randomForest   0.8370786516853933/0.8314606741573034(add min)  0.69138/0.78468(min)
rf_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./rf_fold{i}.pkl')
    rf_pred += model.predict(test2) / 5

    
rf_pred = rf_pred.round().astype(int)

In [ ]:
#get the mean of best fold in decisionTree   0.8258426966292135   0.78947 selected nice
dt_pred = np.zeros(418)
for i in range(5):
    model = joblib.load(f'./dt_fold{i}.pkl')
    dt_pred += model.predict(test2) / 5

    
dt_pred = dt_pred.round().astype(int)

#### way 1:

In [ ]:
#combine lgb_pred rf_pred
f_pro = (lgb_pred + xgboost_pred + catboost_pred + logistic_pred)/4
# f_pro = dt_pred

In [ ]:
pred = []
for i in f_pro:
    if i > 0.5:
        pred.append(1)
    else:
        pred.append(0)

#### way 2:

In [251]:
pred2 = []
for i in range(418):
    count_0 = 0
    count_1 = 0
    for model in [lgb_pred, xgboost_pred, catboost_pred, dt_pred, logistic_pred]:
        if model[i] == 0:
            count_0 += 1
        else:
            count_1 += 1
            
    if count_0 >= count_1:
        pred2.append(0)
    else:
        pred2.append(1)
# print(pred2)   

In [ ]:
sub.Survived = pred

In [ ]:
sub

In [355]:
sub.to_csv('submission_mean_model4.csv', index=False)